In [0]:
import os.path
import pandas as pd
import numpy as np
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
storageAccount = "gen10datafund2111"
storageContainer = "jadr-health-insights"
clientSecret = dbutils.secrets.get(scope = "jadr_blob", key = "clientSecret")
clientid = dbutils.secrets.get(scope = "jadr_blob", key = "clientid")
mount_point = "/mnt/abbeyguilliat/jadr-health"

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": clientid,
       "fs.azure.account.oauth2.client.secret": clientSecret,
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try: 
    dbutils.fs.unmount(mount_point)
except:
    pass

dbutils.fs.mount(
source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/",
mount_point = mount_point,
extra_configs = configs)

display(dbutils.fs.ls("/mnt/abbeyguilliat/jadr-health"))

/mnt/abbeyguilliat/jadr-health has been unmounted.

path,name,size,modificationTime
dbfs:/mnt/abbeyguilliat/jadr-health/Data/,Data/,0,1643742636000
dbfs:/mnt/abbeyguilliat/jadr-health/ML-Models/,ML-Models/,0,1643906451000
dbfs:/mnt/abbeyguilliat/jadr-health/deleteme.txt,deleteme.txt,8,1643742578000


In [0]:
display(dbutils.fs.ls("/mnt/abbeyguilliat/jadr-health/Data/SBOCS2012.SB1200CSA05_2022-01-31T134333"))

path,name,size,modificationTime
dbfs:/mnt/abbeyguilliat/jadr-health/Data/SBOCS2012.SB1200CSA05_2022-01-31T134333/SBOCS2012.SB1200CSA05_data_with_overlays_2022-01-31T133619.csv,SBOCS2012.SB1200CSA05_data_with_overlays_2022-01-31T133619.csv,47299655,1643742881000
dbfs:/mnt/abbeyguilliat/jadr-health/Data/SBOCS2012.SB1200CSA05_2022-01-31T134333/SBOCS2012.SB1200CSA05_metadata_2022-01-31T133619.csv,SBOCS2012.SB1200CSA05_metadata_2022-01-31T133619.csv,1714,1643742862000
dbfs:/mnt/abbeyguilliat/jadr-health/Data/SBOCS2012.SB1200CSA05_2022-01-31T134333/SBOCS2012.SB1200CSA05_table_title_2022-01-31T133619.txt,SBOCS2012.SB1200CSA05_table_title_2022-01-31T133619.txt,7179,1643742862000


In [0]:
census_data = spark.read.options(inferSchema='True',delimiter=',', header='True').csv("/mnt/abbeyguilliat/jadr-health/Data/SBOCS2012.SB1200CSA05_2022-01-31T134333/SBOCS2012.SB1200CSA05_data_with_overlays_2022-01-31T133619.csv")
display(census_data)

GEO_ID,GEO_ID_F,NAME,NAICS2012,NAICS2012_F,NAICS2012_LABEL,SEX,SEX_LABEL,RCPSZFI,RCPSZFI_LABEL,YEAR,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,PAYANN,FIRMNOPD,RCPNOPD,FIRMALL_S,RCPALL_S,FIRMPDEMP_S,RCPPDEMP_S,EMP_S,PAYANN_S,FIRMNOPD_S,RCPNOPD_S
id,Geo Footnote,Geographic Area Name,2012 NAICS code,2012 NAICS Footnote,Meaning of NAICS code,Sex code,Meaning of Sex code,"Sales, value of shipments, or revenue size of firms code","Meaning of Sales, value of shipments, or revenue size of firms code",Year,Number of firms with or without paid employees,"Sales, receipts, or value of shipments of firms with or without paid employees ($1,000)",Number of firms with paid employees,"Sales, receipts, or value of shipments of firms with paid employees ($1,000)",Number of paid employees for pay period including March 12,"Annual payroll ($1,000)",Number of firms without paid employees,"Sales, receipts, or value of shipments of firms without paid employees ($1,000)",Relative standard error of number of firms with or without paid employees (%),"Relative standard error of sales, receipts, or value of shipments of firms with or without paid employees (%)",Relative standard error of number of firms with paid employees (%),"Relative standard error of sales, receipts, or value of shipments of firms with paid employees (%)",Relative standard error of number of paid employees for pay period including March 12 (%),Relative standard error of annual payroll (%),Relative standard error of number of firms without paid employees (%),"Relative standard error of sales, receipts, or value of shipments of firms without paid employees (%)"
0500000US01001,null,"Autauga County, Alabama",00,null,Total for all sectors,001,All firms,001,All firms,2012,2949,2230914,749,2135677,8657,255339,2200,95237,8.4,13.1,14.1,14.5,6.9,8.0,10.8,23.7
0500000US01001,null,"Autauga County, Alabama",00,null,Total for all sectors,002,Female-owned,001,All firms,2012,1093,91041,123,77504,542,14765,970,13537,10.4,19.1,25.3,22.3,31.2,40.2,10.5,21.7
0500000US01001,null,"Autauga County, Alabama",00,null,Total for all sectors,003,Male-owned,001,All firms,2012,1499,710904,371,640410,2943,74908,1128,70493,9.9,39.4,16.4,46.7,17.5,24.6,13.2,33.4
0500000US01001,null,"Autauga County, Alabama",00,null,Total for all sectors,004,Equally male-/female-owned,001,All firms,2012,193,86778,90,75571,788,14444,103,S,35.0,26.8,43.2,31.8,26.6,26.7,48.6,S
0500000US01001,null,"Autauga County, Alabama",00,null,Total for all sectors,096,"All firms classifiable by gender, ethnicity, race, and veteran status",001,All firms,2012,2784,888723,584,793486,4273,104118,2200,95237,8.7,32.3,15.7,38.4,14.0,19.0,10.8,23.7
0500000US01001,null,"Autauga County, Alabama",00,null,Total for all sectors,098,"Publicly held and other firms not classifiable by gender, ethnicity, race, and veteran status",001,All firms,2012,165,1342191,165,1342191,4384,151221,0,0,12.6,4.3,13.1,4.5,2.4,4.2,0.0,0.0
0500000US01001,null,"Autauga County, Alabama",23,null,Construction,001,All firms,001,All firms,2012,406,S,58,S,20 to 99 employees,S,348,S,31.6,S,74.7,S,S,S,33.8,S
0500000US01001,null,"Autauga County, Alabama",23,null,Construction,003,Male-owned,001,All firms,2012,356,S,28,S,20 to 99 employees,S,329,S,35.2,S,86.8,S,S,S,35.7,S
0500000US01001,null,"Autauga County, Alabama",23,null,Construction,004,Equally male-/female-owned,001,All firms,2012,25,S,25,S,20 to 99 employees,S,0,S,122.5,S,127.5,S,S,S,0.0,S


In [0]:
# using pandas to get a better sense/overview of the data
pd_census = census_data.toPandas()
pd_census.drop([0],axis=0,inplace=True)
pd_census = pd_census[pd_census['SEX'] == '001']
pd_census.reset_index(inplace=True)
pd_census

Out[11]:

,index,GEO_ID,GEO_ID_F,NAME,NAICS2012,NAICS2012_F,NAICS2012_LABEL,SEX,SEX_LABEL,RCPSZFI,RCPSZFI_LABEL,YEAR,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,PAYANN,FIRMNOPD,RCPNOPD,FIRMALL_S,RCPALL_S,FIRMPDEMP_S,RCPPDEMP_S,EMP_S,PAYANN_S,FIRMNOPD_S,RCPNOPD_S
0,1,0500000US01001,null,"Autauga County, Alabama",00,null,Total for all sectors,001,All firms,001,All firms,2012,2949,2230914,749,2135677,8657,255339,2200,95237,8.4,13.1,14.1,14.5,6.9,8.0,10.8,23.7
1,7,0500000US01001,null,"Autauga County, Alabama",23,null,Construction,001,All firms,001,All firms,2012,406,S,58,S,20 to 99 employees,S,348,S,31.6,S,74.7,S,S,S,33.8,S
2,11,0500000US01001,null,"Autauga County, Alabama",31-33,null,Manufacturing,001,All firms,001,All firms,2012,67,723131,29,721137,1009,64581,38,1994,49.4,8.2,50.9,8.6,13.1,12.1,65.8,32.1
3,14,0500000US01001,null,"Autauga County, Alabama",42,null,Wholesale trade,001,All firms,001,All firms,2012,78,S,45,S,20 to 99 employees,S,33,S,41.1,S,62.5,S,S,S,41.8,S
4,17,0500000US01001,null,"Autauga County, Alabama",44-45,null,Retail trade,001,All firms,001,All firms,2012,395,755174,166,747240,2532,61278,228,7933,21.3,36.6,20.9,38.5,12.0,22.1,32.9,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43647,179786,0500000US56045,null,"Weston County, Wyoming",54,null,"Professional, scientific, and technical services",001,All firms,001,All firms,2012,52,1704,8,S,20 to 99 employees,S,44,540,40.6,43.6,62.6,S,S,S,43.0,32.8
43648,179789,0500000US56045,null,"Weston County, Wyoming",56,null,Administrative and support and waste managemen...,001,All firms,001,All firms,2012,48,20496,4,20244,70,2981,44,S,106.6,6.9,0.0,7.2,10.1,7.1,110.9,S
43649,179792,0500000US56045,null,"Weston County, Wyoming",62,null,Health care and social assistance,001,All firms,001,All firms,2012,46,7734,11,6900,127,3282,34,S,62.7,12.1,33.2,6.2,6.8,1.9,80.5,S
43650,179795,0500000US56045,null,"Weston County, Wyoming",72,null,Accommodation and food services,001,All firms,001,All firms,2012,33,12488,30,12354,225,3052,3,134,28.1,45.2,32.2,47.5,43.0,51.6,0.0,3.9


In [0]:
#pd_census['RCPSZFI_LABEL'].value_counts()
# drop ['GEO_ID_F'] -- all null
# drop ['NAICS2017_F'] -- we have label
# drop ['index'] -- already have dataframe index
# drop ['SEX','SEX_LABEL','RCPNOPD_S','FIRMNOPD_S','PAYANN_S','EMP_S','RCPPDEMP_S','FIRMPDEMP_S','RCPALL_S','FIRMALL_S','NAICS2012','NAICS2012_F','RCPSZFI_LABEL','RCPSZFI','PAYANN','YEAR']
pd_census.drop(['GEO_ID','SEX','SEX_LABEL','RCPNOPD_S','FIRMNOPD_S','PAYANN_S','EMP_S','RCPPDEMP_S','FIRMPDEMP_S','RCPALL_S','FIRMALL_S','NAICS2012','RCPSZFI_LABEL','RCPSZFI','PAYANN','index','GEO_ID_F','NAICS2012_F'],axis=1,inplace=True)
pd_census

Out[12]:

,NAME,NAICS2012_LABEL,YEAR,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,FIRMNOPD,RCPNOPD
0,"Autauga County, Alabama",Total for all sectors,2012,2949,2230914,749,2135677,8657,2200,95237
1,"Autauga County, Alabama",Construction,2012,406,S,58,S,20 to 99 employees,348,S
2,"Autauga County, Alabama",Manufacturing,2012,67,723131,29,721137,1009,38,1994
3,"Autauga County, Alabama",Wholesale trade,2012,78,S,45,S,20 to 99 employees,33,S
4,"Autauga County, Alabama",Retail trade,2012,395,755174,166,747240,2532,228,7933
...,...,...,...,...,...,...,...,...,...,...
43647,"Weston County, Wyoming","Professional, scientific, and technical services",2012,52,1704,8,S,20 to 99 employees,44,540
43648,"Weston County, Wyoming",Administrative and support and waste managemen...,2012,48,20496,4,20244,70,44,S
43649,"Weston County, Wyoming",Health care and social assistance,2012,46,7734,11,6900,127,34,S
43650,"Weston County, Wyoming",Accommodation and food services,2012,33,12488,30,12354,225,3,134


In [0]:
# ['NAICS2017_LABEL'] == 'Total for all sectors'
pd_census[['COUNTY','STATE']] = pd_census.NAME.str.split(', ',expand=True)
pd_census = pd_census[['COUNTY','STATE','YEAR','NAICS2012_LABEL','FIRMALL','RCPALL','FIRMPDEMP','RCPPDEMP','EMP','FIRMNOPD','RCPNOPD']]
pd_census

Out[13]:

,COUNTY,STATE,YEAR,NAICS2012_LABEL,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,FIRMNOPD,RCPNOPD
0,Autauga County,Alabama,2012,Total for all sectors,2949,2230914,749,2135677,8657,2200,95237
1,Autauga County,Alabama,2012,Construction,406,S,58,S,20 to 99 employees,348,S
2,Autauga County,Alabama,2012,Manufacturing,67,723131,29,721137,1009,38,1994
3,Autauga County,Alabama,2012,Wholesale trade,78,S,45,S,20 to 99 employees,33,S
4,Autauga County,Alabama,2012,Retail trade,395,755174,166,747240,2532,228,7933
...,...,...,...,...,...,...,...,...,...,...,...
43647,Weston County,Wyoming,2012,"Professional, scientific, and technical services",52,1704,8,S,20 to 99 employees,44,540
43648,Weston County,Wyoming,2012,Administrative and support and waste managemen...,48,20496,4,20244,70,44,S
43649,Weston County,Wyoming,2012,Health care and social assistance,46,7734,11,6900,127,34,S
43650,Weston County,Wyoming,2012,Accommodation and food services,33,12488,30,12354,225,3,134


In [0]:
pd_census.drop_duplicates(inplace=True)
pd_census.dropna(how='all',inplace=True)
pd_census.reset_index(inplace=True)
pd_census['RCPALL'].replace('S', 0,inplace=True)
pd_census['RCPPDEMP'].replace('S', 0,inplace=True)
pd_census['RCPNOPD'].replace('S', 0,inplace=True)
pd_census['EMP'].replace('0 to 19 employees','10',inplace=True)
pd_census['EMP'].replace('20 to 99 employees','59',inplace=True)
pd_census['EMP'].replace('100 to 249 employees','175',inplace=True)
pd_census['EMP'].replace('250 to 499 employees','375',inplace=True)
pd_census['EMP'].replace('500 to 999 employees','750',inplace=True)
pd_census['EMP'].replace('1,000 to 2,499 employees','1750',inplace=True)
pd_census['EMP'].replace('2,500 to 4,999 employees','3750',inplace=True)
pd_census['EMP'].replace('5,000 to 9,999 employees','7500',inplace=True)
pd_census['EMP'].replace('10,000 to 24,999 employees','17500',inplace=True)
pd_census['EMP'].replace('25,000 to 49,999 employees','37500',inplace=True)
pd_census['EMP'].replace('50,000 to 99,999 employees','75000',inplace=True)

# stringlen = 6
# for i in range(0,len(pd_census)):
#     x = pd_census['EMP'][i]
#     if len(x) > stringlen:
#         print(x)

<command-522247351783703>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pd_census.drop_duplicates(inplace=True)
<command-522247351783703>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pd_census.dropna(how='all',inplace=True)
/databricks/python/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 return super().replace(

In [0]:
pd_census['FIRMALL'] = pd_census['FIRMALL'].astype('int')
pd_census['FIRMPDEMP'] = pd_census['FIRMPDEMP'].astype('int')
pd_census['FIRMNOPD'] = pd_census['FIRMNOPD'].astype('int')
pd_census['RCPALL'] = pd_census['RCPALL'].astype('int')
pd_census['RCPPDEMP'] = pd_census['RCPPDEMP'].astype('int')
pd_census['RCPNOPD'] = pd_census['RCPNOPD'].astype('int')
pd_census['YEAR'] = pd_census['YEAR'].astype('int')
pd_census['EMP'] = pd_census['EMP'].astype('int')
pd_census = pd_census.replace(0,np.nan)
pd_census.dtypes

Out[15]: index float64
COUNTY object
STATE object
YEAR int64
NAICS2012_LABEL object
FIRMALL int64
RCPALL float64
FIRMPDEMP float64
RCPPDEMP float64
EMP float64
FIRMNOPD float64
RCPNOPD float64
dtype: object

In [0]:
pd_census.reset_index(inplace=True)
for i in range(0,len(pd_census)):
    pd_census['COUNTY'][i] = pd_census['COUNTY'][i][:-7]
pd_census

<command-522247351783713>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pd_census['COUNTY'][i] = pd_census['COUNTY'][i][:-7]
Out[16]:

,level_0,index,COUNTY,STATE,YEAR,NAICS2012_LABEL,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,FIRMNOPD,RCPNOPD
0,0,NaN,Autauga,Alabama,2012,Total for all sectors,2949,2230914.0,749.0,2135677.0,8657.0,2200.0,95237.0
1,1,1.0,Autauga,Alabama,2012,Construction,406,NaN,58.0,NaN,59.0,348.0,NaN
2,2,2.0,Autauga,Alabama,2012,Manufacturing,67,723131.0,29.0,721137.0,1009.0,38.0,1994.0
3,3,3.0,Autauga,Alabama,2012,Wholesale trade,78,NaN,45.0,NaN,59.0,33.0,NaN
4,4,4.0,Autauga,Alabama,2012,Retail trade,395,755174.0,166.0,747240.0,2532.0,228.0,7933.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43647,43647,43647.0,Weston,Wyoming,2012,"Professional, scientific, and technical services",52,1704.0,8.0,NaN,59.0,44.0,540.0
43648,43648,43648.0,Weston,Wyoming,2012,Administrative and support and waste managemen...,48,20496.0,4.0,20244.0,70.0,44.0,NaN
43649,43649,43649.0,Weston,Wyoming,2012,Health care and social assistance,46,7734.0,11.0,6900.0,127.0,34.0,NaN
43650,43650,43650.0,Weston,Wyoming,2012,Accommodation and food services,33,12488.0,30.0,12354.0,225.0,3.0,134.0


In [0]:
pd_census.drop(['level_0','index'],axis=1,inplace=True)
# #census_california.reset_index(inplace=True)
# max_county = 0
# county = ''
# max_label = 0
# label = ''

# for i in range(0,len(pd_census)):
#     x = pd_census['COUNTY'][i]
#     z = pd_census['NAICS2012_LABEL'][i]
#     if len(x) > max_county:
#         max_county = len(pd_census['COUNTY'][i])
#         county = pd_census['COUNTY'][i]
#     if len(z):
#         max_label = len(pd_census['NAICS2012_LABEL'][i])
#         label = pd_census['NAICS2012_LABEL'][i]
# print('max county: ' + county + ' , ' + str(max_county))
# print('max label: ' + label + ' , ' + str(max_label))

# pd_census.shape # 43652 rows
# pd_census.rename(columns={'level_0':'ID'},inplace=True)
# pd_census['ID'] = pd_census['ID'] + 1
# pd_census

In [0]:
# setting up state data frame to load to SQL
states = pd_census[['STATE']]
states = states.drop_duplicates()
states.reset_index(inplace=True)

state_abbr = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

states = states.sort_values(by='STATE')
states['STATE_ABBR'] = state_abbr
states = states[['STATE_ABBR','STATE']]
states

Out[18]:

,STATE_ABBR,STATE
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California
5,CO,Colorado
6,CT,Connecticut
7,DE,Delaware
8,DC,District of Columbia
9,FL,Florida


In [0]:
database = "Jadr-SQL-Database"
table_state = "dbo.State"
table_county = 'dbo.County'
table_census = 'dbo.CensusIndustryData'
user = dbutils.secrets.get(scope = "jadr_blob", key = "SQLUser_dg")
password = dbutils.secrets.get(scope = "jadr_blob", key = "SQLPassword_dg")
server = "gen10-data-fundamentals-21-11-sql-server.database.windows.net"

states['STATE_NAME'] = states['STATE']
states = states[['STATE_ABBR','STATE_NAME']]
spark_states = spark.createDataFrame(states)
display(spark_states)

STATE_ABBR,STATE_NAME
AL,Alabama
AK,Alaska
AZ,Arizona
AR,Arkansas
CA,California
CO,Colorado
CT,Connecticut
DE,Delaware
DC,District of Columbia
FL,Florida


In [0]:
# write states to state SQL table to assign STATE_ID foreign keys
# add try/except statements

# # try:
# #     spark_states.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
# #         .mode("append") \
# #         .option("dbtable", table_state) \
# #         .option("user", user) \
# #         .option("password", password) \
# #         .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
# #         .save()
# # except Exception as e:
# #     print(e)
    
# loaded_states = spark.read.format('jdbc').option("url",f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
#     .option("dbtable", table_state) \
#     .option("user", user) \
#     .option("password", password) \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .load()
    
# loaded_states = loaded_states[['STATE_ABBR','STATE_NAME']]
# #display(loaded_states)
# states_toload = spark_states.subtract(loaded_states)
# #display(states_toload)
    
# states_toload.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
#     .mode("append") \
#     .option("dbtable", table_state) \
#     .option("user", user) \
#     .option("password", password) \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .save()

In [0]:
# read in state SQL table to get STATE_ID foreign keys
spark_state_id = spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table_state) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

display(spark_state_id)

STATE_ID,STATE_ABBR,STATE_NAME
1,MT,Montana
2,NC,North Carolina
3,MD,Maryland
4,CO,Colorado
5,CT,Connecticut
6,IL,Illinois
7,WY,Wyoming
8,NJ,New Jersey
9,DE,Delaware
10,DC,District of Columbia


In [0]:
# convert STATE column in main data frame to corresponding STATE_ID
state_id = spark_state_id.toPandas()
for i in range(0,len(pd_census)):
    for j in range(0,len(state_id)):
        if pd_census['STATE'][i] == state_id['STATE_NAME'][j]:
            pd_census['STATE'][i] = state_id['STATE_ID'][j]
            
pd_census

<command-1536955998085605>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pd_census['STATE'][i] = state_id['STATE_ID'][j]
Out[21]:

,COUNTY,STATE,YEAR,NAICS2012_LABEL,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,FIRMNOPD,RCPNOPD
0,Autauga,28,2012,Total for all sectors,2949,2230914.0,749.0,2135677.0,8657.0,2200.0,95237.0
1,Autauga,28,2012,Construction,406,NaN,58.0,NaN,59.0,348.0,NaN
2,Autauga,28,2012,Manufacturing,67,723131.0,29.0,721137.0,1009.0,38.0,1994.0
3,Autauga,28,2012,Wholesale trade,78,NaN,45.0,NaN,59.0,33.0,NaN
4,Autauga,28,2012,Retail trade,395,755174.0,166.0,747240.0,2532.0,228.0,7933.0
...,...,...,...,...,...,...,...,...,...,...,...
43647,Weston,7,2012,"Professional, scientific, and technical services",52,1704.0,8.0,NaN,59.0,44.0,540.0
43648,Weston,7,2012,Administrative and support and waste managemen...,48,20496.0,4.0,20244.0,70.0,44.0,NaN
43649,Weston,7,2012,Health care and social assistance,46,7734.0,11.0,6900.0,127.0,34.0,NaN
43650,Weston,7,2012,Accommodation and food services,33,12488.0,30.0,12354.0,225.0,3.0,134.0


In [0]:
#state_id.dtypes

In [0]:
# set up county dataframe + replace STATE_NAME w/ STATE_ID
#pd_census.drop(['level_0'],axis=1,inplace=True)
county = pd_census[['COUNTY','STATE']]
county = county.drop_duplicates()
county.reset_index(inplace=True)
county.drop(['index'],axis=1,inplace=True)
# county['STATE_ID'] = 0

# for i in range(0,len(county)):
#     for j in range(0,len(state_id)):
#         if county['STATE'][i] == state_id['STATE_'][j]:
#             print('hi')
#             county['STATE_ID'][i] = state_id['STATE_ID'][j]

#county = county[['COUNTY','STATE_ID']]
county['STATE_ID'] = county['STATE']
county['COUNTY_NAME'] = county['COUNTY']
county = county[['COUNTY_NAME','STATE_ID']]
county

Out[23]:

,COUNTY_NAME,STATE_ID
0,Autauga,28
1,Baldwin,28
2,Barbour,28
3,Bibb,28
4,Blount,28
...,...,...
3134,Sweetwater,7
3135,Teton,7
3136,Uinta,7
3137,Washakie,7


In [0]:
# cali_county = county[county['STATE_ID'] == 15]
# county['STATE_ID'].value_counts()

In [0]:
spark_county = spark.createDataFrame(county)
print(spark_county.count())
display(spark_county)

3139

COUNTY_NAME,STATE_ID
Autauga,28
Baldwin,28
Barbour,28
Bibb,28
Blount,28
Bullock,28
Butler,28
Calhoun,28
Chambers,28
Cherokee,28


In [0]:
# loaded_county = spark.read.format('jdbc').option("url",f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
#     .option("dbtable", table_county) \
#     .option("user", user) \
#     .option("password", password) \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .load()
    
# loaded_county = loaded_county[['COUNTY_NAME','STATE_ID']]
# county_toload = spark_county.subtract(loaded_county)
    
# county_toload.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
#     .mode("append") \
#     .option("dbtable", table_county) \
#     .option("user", user) \
#     .option("password", password) \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .save()

In [0]:
# read in county SQL table to assign foreign keys in main data frame
spark_county_id = spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table_county) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

display(spark_county_id)

COUNTY_ID,COUNTY_NAME,STATE_ID
1,Tuscaloosa,28
2,Sumter,28
3,Fremont,22
4,Pratt,42
5,Baltimo,3
6,Oregon,43
7,Carroll,20
8,Warren,19
9,Cattaraugus,19
10,Jefferson,26


In [0]:
# match up foreign keys in main database!
county_id = spark_county_id.toPandas()

for i in range(0,len(pd_census)):
    for j in range(0,len(county_id)):
        if (pd_census['STATE'][i] == county_id['STATE_ID'][j]) and (pd_census['COUNTY'][i] == county_id['COUNTY_NAME'][j]):
            pd_census['COUNTY'][i] = county_id['COUNTY_ID'][j]
            
#pd_census.drop(['STATE'],axis=1,inplace=True)
pd_census

<command-1536955998085631>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pd_census['COUNTY'][i] = county_id['COUNTY_ID'][j]
Out[26]:

,COUNTY,STATE,YEAR,NAICS2012_LABEL,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,FIRMNOPD,RCPNOPD
0,172,28,2012,Total for all sectors,2949,2230914.0,749.0,2135677.0,8657.0,2200.0,95237.0
1,172,28,2012,Construction,406,NaN,58.0,NaN,59.0,348.0,NaN
2,172,28,2012,Manufacturing,67,723131.0,29.0,721137.0,1009.0,38.0,1994.0
3,172,28,2012,Wholesale trade,78,NaN,45.0,NaN,59.0,33.0,NaN
4,172,28,2012,Retail trade,395,755174.0,166.0,747240.0,2532.0,228.0,7933.0
...,...,...,...,...,...,...,...,...,...,...,...
43647,566,7,2012,"Professional, scientific, and technical services",52,1704.0,8.0,NaN,59.0,44.0,540.0
43648,566,7,2012,Administrative and support and waste managemen...,48,20496.0,4.0,20244.0,70.0,44.0,NaN
43649,566,7,2012,Health care and social assistance,46,7734.0,11.0,6900.0,127.0,34.0,NaN
43650,566,7,2012,Accommodation and food services,33,12488.0,30.0,12354.0,225.0,3.0,134.0


In [0]:
# pd_census.replace(np.nan,0,inplace=True)
pd_census.info()

In [0]:
pd_census['COUNTY_ID'] = pd_census['COUNTY']
pd_census = pd_census[['COUNTY_ID','YEAR','NAICS2012_LABEL','FIRMALL','RCPALL','FIRMPDEMP','RCPPDEMP','EMP','FIRMNOPD','RCPNOPD']]
pd_census['COUNTY_ID'] = pd_census['COUNTY_ID'].astype('int')
spark_census = spark.createDataFrame(pd_census)
display(spark_census)

<command-1536955998085636>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pd_census['COUNTY_ID'] = pd_census['COUNTY_ID'].astype('int')

COUNTY_ID,YEAR,NAICS2012_LABEL,FIRMALL,RCPALL,FIRMPDEMP,RCPPDEMP,EMP,FIRMNOPD,RCPNOPD
172,2012,Total for all sectors,2949,2230914.0,749.0,2135677.0,8657.0,2200.0,95237.0
172,2012,Construction,406,null,58.0,null,59.0,348.0,null
172,2012,Manufacturing,67,723131.0,29.0,721137.0,1009.0,38.0,1994.0
172,2012,Wholesale trade,78,null,45.0,null,59.0,33.0,null
172,2012,Retail trade,395,755174.0,166.0,747240.0,2532.0,228.0,7933.0
172,2012,Transportation and warehousing,87,16962.0,10.0,null,59.0,77.0,4977.0
172,2012,Information,27,22786.0,10.0,21362.0,77.0,17.0,1425.0
172,2012,Finance and insurance,109,82390.0,47.0,71087.0,407.0,63.0,null
172,2012,Real estate and rental and leasing,87,13387.0,14.0,7833.0,63.0,73.0,null
172,2012,"Professional, scientific, and technical services",217,45988.0,59.0,null,375.0,158.0,null


In [0]:
print(spark_census.count())

In [0]:
# write data main data frame to census SQL table !!!
spark_census.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table_census) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()